In [1]:
import tensorflow as tf
import os 
import glob
import numpy as np

In [4]:
tf.config.run_functions_eagerly(True)

class TFrecordDataLoader:
    def __init__(self, datadir, batch_size=64, buffer_size=20000, img_size=224):
        self.datadir = datadir
        self.tfrecords = glob.glob(f"{datadir}/*/*.tfrecord")
        self.label_map = self.create_label_map()
        self.batch_size = batch_size
        self.buffer_size = buffer_size
        self.img_size = (img_size, img_size) 


    def create_label_map(self):
        barcodes = os.listdir(self.datadir)
        label_map = {barcode: index for index, barcode in enumerate(barcodes, 1)}
        return label_map


    def parse_tfrecord_fn(self, example):
        feature_description = {
            'image/height': tf.io.FixedLenFeature([], tf.int64),
            'image/width': tf.io.FixedLenFeature([], tf.int64),
            'image/channels': tf.io.FixedLenFeature([], tf.int64),
            'image/colorspace': tf.io.FixedLenFeature([], tf.string),
            'image/class/text': tf.io.FixedLenFeature([], tf.string),
            'image/format': tf.io.FixedLenFeature([], tf.string),
            'image/filename': tf.io.FixedLenFeature([], tf.string),
            'image/encoded': tf.io.FixedLenFeature([], tf.string),
            # Add more features as needed, specify their types and shapes
        }
        example = tf.io.parse_single_example(example, feature_description)
        return example
    
    def parse_label(self, barcode):
        label = self.label_map[barcode.numpy().decode('utf-8')]
        return label 
    

    def parse_example(self, example):
        image = example['image/encoded']
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.cast(image, tf.float32) / 255.0
        image = tf.image.resize(image, self.img_size)
        label = tf.py_function(self.parse_label, [example['image/class/text']], tf.int32)
        label = tf.one_hot(label, len(self.label_map))
        return image, lable
    

    def load_dataset(self):
        dataset = tf.data.TFRecordDataset(self.tfrecords)
        dataset = dataset.map(self.parse_tfrecord_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        dataset = dataset.map(self.parse_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        dataset = dataset.shuffle(buffer_size=self.buffer_size)
        dataset = dataset.batch(self.batch_size)
#         dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
        return dataset

In [3]:
datadir = "/data/fourview_cam0_tfrecord"
batch_size = 128
data_loader = TFrecordDataLoader(datadir, batch_size=batch_size)
dataset = data_loader.load_dataset()

AttributeError: in user code:

    <ipython-input-2-1446073d25ae>:44 parse_example  *
        return image.numpy(), lable.numpy()

    AttributeError: 'Tensor' object has no attribute 'numpy'


In [ ]:
for images, labels in dataset:
    print(images.shape)
    print(images)
    print(labels.shape)
    print(labels)

In [41]:
IMAGE_SIZE = 224
NUM_CLASSES = len(os.listdir(datadir))
strategy = tf.distribute.MirroredStrategy()
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
print(NUM_CLASSES)
print(IMG_SHAPE)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
753
(224, 224, 3)


In [42]:
with strategy.scope():
    base_model = tf.keras.applications.MobileNet(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')
#     base_model.trainable = False
    model = tf.keras.Sequential([
          base_model,
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(units=1024, activation='relu'),
          tf.keras.layers.Dense(units=512, activation='relu'),
          tf.keras.layers.Dense(units=NUM_CLASSES, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam', 
        loss='categorical_crossentropy', 
        metrics=['accuracy'])

In [43]:
with strategy.scope():
    history = model.fit(dataset, epochs=10)

Epoch 1/10
INFO:tensorflow:batch_all_reduce: 87 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Error reported to Coordinator: as_list() is not defined on an unknown TensorShape.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/training/coordinator.py", line 297, in stop_on_exception
    yield
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/mirrored_run.py", line 323, in run
    self.main_result = self.main_fn(*self.main_args, **self.main_kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py", line 255, in wrapper
    return converted_call(f, args, kwargs, options=options)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py", line 532, in converted_call
    return _call_unconverted(f, args, kwargs, options)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/autograph/impl/api.py", line 339, in _call_unc

ValueError: in user code:

    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/mirrored_strategy.py:585 _call_for_each_replica
        self._container_strategy(), fn, args, kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/mirrored_run.py:96 call_for_each_re